In [2]:
import os
import json
import requests

from transformers import AutoConfig
from transformers import AutoTokenizer

from sentence_transformers import util
from sentence_transformers import SentenceTransformer

In [6]:
#json string data
employee_string = '[{"first_name": "Michael", "last_name": "Rodgers", "department": "Marketing"}]'

#check data type with type() method
print(type(employee_string))

#convert string to  object
json_object = json.loads(employee_string)

<class 'str'>


In [10]:
history = []
history.append({"role": "user", "content": "valami"})
history = json.dumps(history)
print(history)
json.loads(history)

[{"role": "user", "content": "valami"}]


[{'role': 'user', 'content': 'valami'}]

In [295]:
def question_checkker(question, refq, treshold=0.5, model=None):
    if model is not None:
        emb_refq = model.encode(refq)
        emb_actq = model.encode([question])

        cosine_similarities = util.cos_sim(emb_refq, emb_actq)

        return cosine_similarities.mean() > treshold
    else:
        return False

In [296]:
def cleaning_stream(batch):
    if len(batch['text_output']) == 0:
        return False
    elif batch['text_output'] == "\n\n":
        return False
    elif "<|start_header_id|>" in batch['text_output']:
        return False
    elif "assistant" in batch['text_output']:
        return False
    elif "<|end_header_id|>" in batch['text_output']:
        return False
    else:
        return True

In [334]:
with open('config.json', 'r') as file:
    config = json.load(file)

In [326]:
with open(f"{config['sources']}/total.json", 'r') as file:
    data = json.load(file)

In [327]:
url = f"{config['triton']['host']}:{config['triton']['port']}/v2/{config['triton']['model']}/{config['triton']['generation']}"
url

'http://172.22.214.120:28800/v2/models/llama-3.2-1b-instruct/generate_stream'

In [330]:
tokenizer = AutoTokenizer.from_pretrained(config["model repository"])
autoconfig = AutoConfig.from_pretrained(config["model repository"])

print(" Model Name\t\t: ", config["model repository"],"\n", "Max token input size:\t: ", autoconfig.max_position_embeddings)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

c:\Users\lrobert\anaconda3\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lrobert\.cache\huggingface\hub\models--meta-llama--Llama-3.2-1b-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

 Model Name		:  meta-llama/Llama-3.2-1b-instruct 
 Max token input size:	:  131072


In [331]:
model_st = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [340]:
# question = "Mennyi volt az átlagos páratartalom az utolsó 24 órában?"
question = "Mennyi volt az átlagos páratartalom 2024.09.16-án 10 órakkor."
question

'Mennyi volt az átlagos páratartalom 2024.09.16-án 10 órakkor.'

In [339]:
if question_checkker(question=question, 
                    refq=config["filters"]["reference questions"], 
                    treshold=config["filters"]["treshold"], 
                    model=model_st):
    print("Ok")
else:
    print(config["guard"]["no information"])

Sajnálom erre a kérdésre nem áll rendelkezésre információ.


In [341]:
reinforce = " Csak a kérdésre válaszolj! Ne magyarázd a válaszod és ne sorolj fel részadatokat de teljes mondatban válaszolj!"
chat = config["chat history"].copy()
chat[-1]["content"] = f"Páratartalom adatai (JSON):{str(data)}\n\n"
chat.append({"role": "user", "content": question + reinforce})

message = tokenizer.apply_chat_template(chat, tokenize=False)

len(tokenizer.encode(message))                            

11248

In [352]:
if autoconfig.max_position_embeddings > len(tokenizer.encode(message)):
    payload = {
        "text_input": message,
        "max_tokens": config['triton']['max_tokens'],
        "temperature": config['triton']['temperature'],
        "stream": ("stream" in config['triton']['generation'])
    }

    # Kérelem küldése és stream feldolgozása
    with requests.post(url, json=payload, stream=True) as response:
        # Ellenőrizzük, hogy sikeres volt-e a válasz
        if response.status_code == 200:
            # Streameljük a válasz sorokat
            for line in response.iter_lines():
                if line:
                    # JSON-adat soronkénti feldolgozása
                    batch = json.loads(line.decode('utf-8').replace("data: ", ""))
                    if cleaning_stream(batch):
                        print(batch['text_output'], end='', flush=True)
                    # print(data['text_output'], end='\n', flush=True)
        else:
            print(f"Hiba történt: {response.status_code}")
else:
    print("Need cutting")

2024.09.12 1 óra/h: '48.33%', 2024.09.13 2 óra/h: '49.22%', 2024.09.14 3 óra/h: '47.914%', 2024.09.15 4 óra/h: '48.215%', 2024.09.16 5 óra/h: '44.258%'

In [16]:
# import requests
# import json

# # A POST URL-je a model híváshoz
# url = "http://172.22.214.120:28800/v2/models/llama-3.2-1b-instruct/generate"
# # url = "http://172.22.214.120:28800/v2/models/llama-3.1-8b-instruct/generate"

# # A kéréshez tartozó payload
# payload = {
#     "text_input": "machine learning is",
#     "max_tokens": 128,
#     "temperature": 0.95
# }

# # Kérelem küldése és a válasz feldolgozása
# response = requests.post(url, json=payload)

# # Ellenőrizzük, hogy sikeres volt-e a válasz
# if response.status_code == 200:
#     # Teljes JSON válasz feldolgozása
#     data = response.json()
#     print(data['text_output'])
# else:
#     print(f"Hiba történt: {response.status_code}")


machine learning is an absolute hero of modern technology. It has numerous applications and transformed the way businesses operate today. Machine learning algorithms have been used to identify defect in products, improves supply chain efficiency by minimizing waste, and even provides personalized recommendations in customer service.

With the rise of artificial intelligence and deep learning, we are witness to a new era of AI automation. This technology is being applied to various industries such as finance, healthcare, and even the delivery of food. For instance, chatbots and virtual assistants are being used to handle customer inquiries, provide healthcare services, and even assist in estimating costs for legal documents.

An extra added benefit is that these artificial
